In [4]:
import pandas as pd
import numpy as np
from numpy.random import randint

target_train = pd.read_csv('train_labels.csv')

sample_submission = pd.read_csv('sample_submission.csv')

df_train = pd.read_csv('train_features.csv')

target_train.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [58]:
print(target_train.shape[0], target_train.id.max(), sample_submission.shape[0])

valuecounts = target_train.status_group.value_counts()
status_group_names = valuecounts.index

predict_list = [(k, status_group_names[randint(0, 2)]) for k in sample_submission.id]

prediction = pd.DataFrame(predict_list, columns=['id', 'status_group'])

prediction.head()

prediction.shape

59400 74247 14358


(14358, 2)

In [57]:
# write to csv and submit

prediction.to_csv('noise_submit.csv', index=False)

!kaggle competitions submit -c ds1-predictive-modeling-challenge -f noise_submit.csv -m "baseline noise"

100%|█████████████████████████████████████████| 265k/265k [00:01<00:00, 269kB/s]
Successfully submitted to DS1 Predictive Modeling Challenge

In [65]:
def write_submit(model, name='submission.csv'): 
    model.to_csv(name, index=False)
    !kaggle competitions submit -c ds1-predictive-modeling-challenge -f submission.csv -m "message"
    pass 



In [32]:
insigs = ['funder', 'installer', 'subvillage', 'scheme_name', 'ward']

# df_train.head()
# for name in insigs:
#     df_train[name].nuniques()

#df_train.select_dtypes(include='object').nunique()

#df_train[['region', 'district_code', 'ward']].nunique()

# df_train['region_district'] = df_train.apply(lambda row: f'{row.region}_{row.district_code}', axis=1)
# df_train['region_district_ward'] = df_train.apply(lambda row: f'{row.region}_{row.district_code}_{row.ward}', axis=1)


#df_train[['region', 'district_code', 'ward', 'region_district', 'region_district_ward']].nunique()

df_train[insigs].nunique()

df_train.select_dtypes(include='object').nunique()

df_train.nunique()

id                       59400
amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_

In [47]:
from time import time
import gc

In [49]:
%%time

gc.enable()
dat = pd.read_csv('test_features.csv')

def insignificant(features, k=4): 
    k = time()
    cumula = 0
    for feat in features:
        j = time()
        cumula += j - k
        print(f'beginning insig on {feat}; at time {cumula:.3}....')

        def new_feat_mapper(x):
            valcountsbool = (dat[feat].value_counts()<=k)
            if x in valcountsbool[valcountsbool.to_numpy() == True].index: 
                return 'OTHER'
            else: 
                return x
        dat[feat] = dat[feat].apply(new_feat_mapper)
        gc.collect()
        continue
    pass

insignificant(insigs)

beginning insig on funder; at time 9.54e-07....
beginning insig on installer; at time 68.0....
beginning insig on subvillage; at time 1.82e+02....


KeyboardInterrupt: 

In [ ]:
## it took 5 minutes total, 

## let's enable garbage collection

## ok gc does NOTHING. 

In [88]:
# insigs = ['funder', 'installer', 'subvillage', 'ward']


dat = pd.read_csv('test_features.csv')
dat = dat.fillna('NOT_KNOWN')
cats = ['region_district'] + list(dat.select_dtypes(include='object').drop(['region'], axis=1).columns)
nums = list(dat.select_dtypes(exclude='object').drop(['id', 'district_code'], axis=1).columns)
boolski = ['public_meeting', 'permit']

to_drop = ['wpt_name', 'region', 'district_code', 'ward', 'scheme_name']

insigs = ['funder', 'installer', 'subvillage']

dat = dat[cats].drop(to_drop + boolski + ['region_district'], axis=1).str.lower()

def insignificant_two(features, k=3): 
    for feat in features: 
        valcts = dat[feat].str.lower().value_counts()

        dat[feat] = [val if valcts[val] > k else "OTHER" for val in dat[feat].str.lower()]
        continue
    pass



insignificant_two(insigs)

dat[insigs]

KeyError: "['region_district'] not in index"